In [1]:
import os
base_dir = './'
non_recyclable_dir = os.path.join(base_dir, 'non-recyclable')
recyclable_dir = os.path.join(base_dir, 'recyclable')
test_dir = os.path.join(base_dir, 'test')
test_nonrecyclable_dir = os.path.join(test_dir, 'test_nonrecyclable')
test_recyclable_dir = os.path.join(test_dir, 'test_recycleable')

In [2]:
print("Non-recyclable images:", len(os.listdir(non_recyclable_dir)))
print("Recyclable images:", len(os.listdir(recyclable_dir)))
print("Test non-recyclable images:", len(os.listdir(test_nonrecyclable_dir)))
print("Test recyclable images:", len(os.listdir(test_recyclable_dir)))

Non-recyclable images: 40
Recyclable images: 40
Test non-recyclable images: 10
Test recyclable images: 10


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 200, 200
batch_size = 10

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    classes=['recyclable', 'non-recyclable'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    base_dir,
    classes=['recyclable', 'non-recyclable'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 80 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

2024-04-28 12:19:02.096365: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-04-28 12:19:02.096394: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-28 12:19:02.096405: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-28 12:19:02.096479: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-28 12:19:02.096526: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=8
)

Epoch 1/10


2024-04-28 12:19:08.289895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 3s 59ms/step - loss: 5.2698 - accuracy: 0.5500 - val_loss: 6.0950 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 0s 35ms/step - loss: 4.9749 - accuracy: 0.4500 - val_loss: 2.4924 - val_accuracy: 0.5000
Epoch 3/10
8/8 [==============================] - 0s 34ms/step - loss: 1.8987 - accuracy: 0.5375 - val_loss: 1.1355 - val_accuracy: 0.5000
Epoch 4/10
8/8 [==============================] - 0s 35ms/step - loss: 0.7637 - accuracy: 0.5750 - val_loss: 0.5381 - val_accuracy: 0.7875
Epoch 5/10
8/8 [==============================] - 0s 35ms/step - loss: 0.5561 - accuracy: 0.7500 - val_loss: 0.4842 - val_accuracy: 0.7750
Epoch 6/10
8/8 [==============================] - 0s 37ms/step - loss: 0.5194 - accuracy: 0.7500 - val_loss: 0.6430 - val_accuracy: 0.6000
Epoch 7/10
8/8 [==============================] - 0s 36ms/step - loss: 0.4810 - accuracy: 0.7000 - val_loss: 0.6724 - val_accuracy: 0.5875
Epoch 8/10
8/8 [======================

In [6]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

2/2 [==============================] - 0s 14ms/step - loss: 1.5119 - accuracy: 0.5000
Test accuracy: 0.5


In [7]:
import numpy as np
from tensorflow.keras.preprocessing import image

def load_and_predict(img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.

    prediction = model.predict(img_tensor)
    return "recyclable" if prediction[0] < 0.5 else "non-recyclable"

test_img_paths = [
    os.path.join(test_recyclable_dir, 'QLCZ2890.jpg'),
    os.path.join(test_nonrecyclable_dir, 'WRAQ6784.jpg') 
]

for path in test_img_paths:
    result = load_and_predict(path)
    print(f'{path}: {result}')


1/1 [==============================] - 0s 364ms/step
./test/test_recycleable/QLCZ2890.jpg: non-recyclable
1/1 [==============================] - 0s 14ms/step
./test/test_nonrecyclable/WRAQ6784.jpg: non-recyclable


In [9]:
filename = 'recycling_classification_model.h5'
model.save(filename)

print(f'Model saved to {filename}')

/Users/adarsh/.pyenv/versions/3.9.18/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to recycling_classification_model.h5
